In [1]:
#importing required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
# Checking the tensorflow version
print(tf.__version__)

2.10.0


In [3]:
import os
os.chdir(r'C:\Deep Learning')

In [4]:
# Loading the data
bank_data = pd.read_csv("Artificial_Neural_Network_Case_Study_data.csv")

# Taking  all rows and all columns in the data except the last column as X (feature matrix)
#the row numbers and customer id's are not necessary for the modelling so we get rid of and start with credit score
X = bank_data.iloc[:,3:-1].values

#taking all rows but only the last column as Y(dependent variable)
y = bank_data.iloc[:, -1].values


In [5]:
bank_data['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [6]:
# Transforming the gender variable, labels are chosen randomly
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [7]:
# Transforming the geography column variable, labels are chosen randomly, the ct asks for argument [1] the index of the target vb
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(),[1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [8]:
# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
#printing the dimensions of each of those snapshots to see amount of rows and columns i each of them
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(8000, 12) (2000, 12)
(8000,) (2000,)


In [9]:
# Data Scaling/normalization of the features that will go to the NN
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
### Model Building

In [11]:
# Initializing the ANN by calling the Sequential class fromm keras of Tensorflow
ann = tf.keras.models.Sequential()

In [12]:
# Adding "fully connected" INPUT layer to the Sequential ANN by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 6 and Activation Function = Rectifier
ann.add(tf.keras.layers.Dense(units = 8, activation = 'sigmoid'))

In [13]:
# Adding "fully connected" SECOND layer to the Sequential AMM by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 6 and Activation Function = Rectifier
ann.add(tf.keras.layers.Dense(units = 6, activation = 'sigmoid'))

In [14]:
# Adding "fully connected" OUTPUT layer to the Sequential ANN by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 1 and Activation Function = Sigmoid
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

In [15]:
#----------------------- Training the model -----------------------#
# Compiling the ANN
# Type of Optimizer = Adam Optimizer, Loss Function =  crossentropy for binary dependent variable, and Optimization is done w.r.t. accuracy
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [16]:
# Training the ANN model on training set  (fit method always the same)
# batch_size = 32, the default value, number of epochs  = 100
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 760us/step - loss: 0.5076 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 771us/step - loss: 0.4793 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 723us/step - loss: 0.4610 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 743us/step - loss: 0.4451 - accuracy: 0.7962
Epoch 5/100
250/250 [==============================] - 0s 741us/step - loss: 0.4345 - accuracy: 0.8019
Epoch 6/100
250/250 [==============================] - 0s 882us/step - loss: 0.4288 - accuracy: 0.8133
Epoch 7/100
250/250 [==============================] - 0s 711us/step - loss: 0.4253 - accuracy: 0.8161
Epoch 8/100
250/250 [==============================] - 0s 740us/step - loss: 0.4232 - accuracy: 0.8163
Epoch 9/100
250/250 [==============================] - 0s 728us/step - loss: 0.4216 - accuracy: 0.8179
Epoch 10/100
250/250 [==============================] - 0s 732us/step - l

In [ ]:
X_test.shape

In [ ]:
ann.get_weights()[4].shape

In [17]:
ann.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (32, 8)                   104       
                                                                 
 dense_1 (Dense)             (32, 6)                   54        
                                                                 
 dense_2 (Dense)             (32, 1)                   7         
                                                                 
Total params: 165
Trainable params: 165
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_pred_prob = ann.predict(X_test)

In [ ]:
y_pred_prob.shape

In [ ]:
#show the vector of predictions and real values
#probabilities
y_pred_prob = ann.predict(X_test)

#probabilities to binary
y_pred = (y_pred_prob > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)), 1))

#Confusion Matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix", confusion_matrix)
print("Accuracy Score", accuracy_score(y_test, y_pred))